In [1]:

# using Pkg
# Pkg.activate("/home/jek354/research/ML-signproblem")
# Pkg.update()

using Lattices
using LinearAlgebra
using Combinatorics
using SparseArrays
using Plots
import Graphs
using LaTeXStrings
using Statistics
using Random
using Zygote
using Optimization, OptimizationOptimisers
using JSON
using OptimizationOptimJL
using JLD2
using ExponentialUtilities


include("ed_objects.jl")
include("ed_functions.jl")
include("ed_optimization.jl")
include("utility_functions.jl")
     

merge_jld2_folder

In [11]:
M = rand(2,1,1)
x = [1.0, 0]
y = [0.0, 1]
# my_matexp(x) = exp(1im * x)
grad_func = COMBDIF_GRAD_GEN(M)
for i=1:2
println(grad_func([1.0 1.4], 1.0 + 0im)(i))
end

0.6292591560527264 + 0.0im
0.5288128503732886 + 0.0im


In [3]:
# using KrylovKit


t = 1.0
U = 6
μ = 0  # positive incentivises fewer particles (one electron costs this much energy)
N_up = 4
N_down = 4
N =  3
half_filling = false
# lattice = Chain(6, Periodic())
lattice_dimension = (2,3)
bc = "periodic"
# lattice = Chain(6, Periodic())
lattice = Square(lattice_dimension, if bc == "periodic" Periodic() else Open() end)
# lattice = Graphs.cycle_graph(3)

models = []

reference_index = 2
# t_values = []# Float64[1,1], Float64[1,0.1], 1.0]
# for _t in t_values
#     # println(_t)
#     push!(models,HubbardModel(_t,0.0001,μ,half_filling))
# end
# U_values = [0.00001,0.01]
U_values = [0.00001; LinRange(2.1,9,20)]
U_values = sort([U_values; 10.0 .^LinRange(-3,2,40)])

for U in U_values
    # println(t)
    push!(models,HubbardModel(t,U,μ,half_filling))
end

subspace = HubbardSubspace(N, lattice)
# subspace = HubbardSubspace(N_up, N_down, lattice)

ops = []
if subspace.N >= 1
    push!(ops,create_operator(subspace,:Sx))
end
push!(ops, create_operator(subspace,:S2))
# op3 = Matrix(create_operator(subspace,:L2))
push!(ops, create_operator(subspace,:T, kind=1))
push!(ops, create_operator(subspace,:T, kind=2))

H = []
for model ∈ models
    push!(H, Matrix(create_Hubbard(model, subspace)))
end


# _, indexer = create_Hubbard(models[1], subspace; get_indexer=true)
# degen_rm_U_1 = create_consistent_basis(H, ops;reference_index=reference_index)

In [3]:
include("ed_objects.jl")
include("ed_functions.jl")
include("ed_optimization.jl")
include("utility_functions.jl")

dic = load_saved_dict("data/N=3_2x3/meta_data_and_E.jld2")

meta_data = dic["meta_data"]
U_values = meta_data["U_values"]
all_full_eig_vecs = dic["all_full_eig_vecs"]
n_eigs = [parse(Int,x) for x in split(meta_data["sites"], "x")]
indexer = dic["indexer"]

# meta_data = Dict("electron count"=>3, "sites"=>"2x3", "bc"=>"periodic", "basis"=>"adiabatic", 
#                 "U_values"=>U_values, "maxiters"=>10)
instructions = Dict("starting state"=>Dict("U index"=>1, "levels"=>1),
                "ending state"=>Dict("U index"=>30, "levels"=>1), "max_order"=>2, "use symmetry"=>false)
println("U in [",U_values[instructions["starting state"]["U index"]], ", ", U_values[instructions["ending state"]["U index"]],"]")

# x = zero(degen_rm_U[1:2,:])
# x[1,1] = 1
# x[2,2] = 1
data_dict_tmp = test_map_to_state(all_full_eig_vecs[6], instructions, indexer, !isa(meta_data["electron count"], Number);
     maxiters=3#meta_data["maxiters"]
     , optimization=:gradient)
data_dict_tmp
# save_with_metadata(data_dict_tmp, "data/tmp.jld2")

U in [1.0e-5, 2.463157894736842]
magnitude: 0.013941348582661217
learning rate: 0.0027882697165322436
  0.113226 seconds (213.08 k allocations: 9.214 MiB, 99.91% compilation time)
  0.008103 seconds (260.49 k allocations: 29.858 MiB, 35.73% compilation time)
0.028320727055685224
0.028320727055685224
0.028164945182476697
0.028164945182476697
0.027990816725704026
0.027990816725704026
0.027927574570635993
0.027927574570635993
0.02790892168073522
0.02790892168073522
0.027891550200981552
0.027891550200981552
0.027890721698462073
0.027890721698462073
  8.331264 seconds (40.47 M allocations: 2.948 GiB, 12.88% gc time, 87.28% compilation time: <1% of which was recompilation)
Finished order 1
loss std: 0.00020008043218715355
magnitude: 0.013945360849231037
learning rate: 0.0027890721698462075
  0.000576 seconds (35.15 k allocations: 2.366 MiB)
  0.099763 seconds (6.11 M allocations: 602.519 MiB, 25.87% gc time)
0.030899507645245072
0.030899507645245072
0.004751825978285273
0.004751825978285273


Dict{String, Any} with 11 entries:
  "loss_metrics"       => Any[Any[0.0278827, 0.0278907, 0.00093974]]
  "param_mapping"      => Any[nothing, nothing]
  "loss_std_metrics"   => Any[Any[0.0, 0.00020008, 0.0142505]]
  "all_matrices"       => Any[Any[sparse([1, 2, 3, 4, 5, 6, 7, 11, 12, 13  …  2…
  "parities"           => Any[nothing, nothing]
  "coefficient_labels" => Any[Vector{Tuple{Coordinate{2, Int64}, Int64, Symbol}…
  "other_metrics"      => Vector{Any}[[ * Status: failure (reached maximum numb…
  "norm2_metrics"      => Any[[0.145939, 1.22021]]
  "norm1_metrics"      => Any[[7.02167, 154.117]]
  "labels"             => Any[Dict("starting state"=>Dict("U index"=>1, "level"…
  "coefficients"       => Any[Any[[-0.000998123, 4.42978e-5, 0.00049719, -0.000…

In [10]:
abs2(all_full_eig_vecs[6][1,:]'*all_full_eig_vecs[6][50,:])

0.8488729446218476

In [107]:
tmp1 = data_dict_tmp["coefficients"]

1-element Vector{Any}:
 Any[[0.00804608274539668, -0.0035627311308626727, 0.013194835069913913, -0.019268140850836987, -0.0014423204630328395, -0.0014497388671231462, -0.000581176258498751, 0.004805596792375161, -0.0006780557731068106, -0.00483712196787348  …  0.0028166715512127115, 0.013390362781723337, 0.0053384682331121735, 0.0010069621609959313, 0.0019766507430968986, -0.005283564663695769, -0.009784655021768559, -0.0010179350165393398, 0.013508137220184915, -0.002029387141122564]]

In [109]:
tmp2 =  data_dict_tmp["coefficients"]

1-element Vector{Any}:
 Any[[0.004102925388190125, 0.006857089225758785, 0.010862586647674527, -0.015113738591211994, 0.0020029422241328717, -0.003917419109382482, 0.0006817613154320765, -0.0027356295135367104, 0.003944283685439162, -0.0011206962068358818  …  -0.00445876724572687, 0.014266343195250178, -0.0008161245510463928, 0.004458669581031902, -0.005829410058194202, -0.005148793353387051, -0.0135012837269036, 0.0014134604142347466, 0.01564486999853579, 0.0001853136543891242]]

In [91]:
h_idx = 1
sort_i = sortperm(real.(diag(degen_rm_U_1[h_idx]'*H[h_idx]*degen_rm_U_1[h_idx])))
# degen_rm_U_1[1][:,sort_i]'*H[1]*degen_rm_U_1[1][:,sort_i]
for i = 1:10
    for k = 1:6
    state1 = degen_rm_U_1[h_idx][:,sort_i[i]]
    state2 = all_full_eig_vecs[k][h_idx,:]
    if abs(state1'*state2) > 0.999 #&& abs(state1'*state2) < 0.9999
        println("$(sort_i[i]) $k")
        println("overlap: ", abs(state1'*state2))
        println("Operator expectation")
        for op in ops
            println(state1'*op*state1)
            println(state2'*op*state2)
            println()
        end
        println("Energy")
        println(state1'*H[h_idx]*state1)
        println(state2'*H[h_idx]*state2)
        println("--------")
    end
    end
end

209 6
overlap: 1.0000000000000004
Operator expectation
-0.5000000000000004 + 3.6485613330733966e-18im
-0.49999999999999994 + 2.125326019440164e-18im

0.7500000000000004 + 5.813531224141085e-18im
0.75 - 1.0918852741690807e-17im

-1.0000000000000009 - 9.697048247126494e-18im
-1.0 + 1.5846320501374278e-17im

-0.5000000000000004 + 0.8660254037844392im
-0.5 + 0.8660254037844388im

Energy
-8.999996666669809 + 1.1245712333560524e-17im
-8.999996666669803 + 1.062207109125475e-17im
--------
106 4
overlap: 1.0
Operator expectation
-0.4999999999999997 + 3.3097316105594254e-18im
-0.5000000000000001 - 8.553055805603911e-19im

0.7499999999999997 - 5.71922932558759e-18im
0.7500000000000002 - 2.646269893565343e-18im

-0.9999999999999997 + 5.152124223788585e-18im
-1.0000000000000002 + 9.103246718237953e-18im

-0.4999999999999997 - 0.8660254037844382im
-0.5000000000000001 - 0.8660254037844388im

Energy
-8.999996666669798 - 5.494215684893975e-17im
-8.999996666669803 + 8.471574875673278e-17im
--------
102 